In [1]:
from bert.utils import obtain_sentence_embeddings
from data.utils import load_training_dictionaries
from extractor.utils import ExtractorModel
from extractor.train import get_training_batch
from pytorch_transformers import BertModel
from pytorch_transformers import BertTokenizer

import torch
import numpy as np

In [2]:
# Load data:
model = ExtractorModel()
model_path = "results/models/extractor.pt"
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [3]:
data = load_training_dictionaries()
documents, extraction_labels = get_training_batch(data, batch_size=10)

In [4]:
sentence_embeddings, mask = obtain_sentence_embeddings(model.bert_model, model.bert_tokenizer, documents)

In [5]:
extraction_probabilities = model(sentence_embeddings)
extraction_probabilities

tensor([[5.9915e-04, 9.9328e-01, 5.3232e-04, 5.4050e-04, 5.6455e-04, 9.9860e-01,
         9.9871e-01, 5.4179e-04, 5.1287e-04, 6.5203e-04, 7.0618e-04, 5.4396e-04,
         5.4196e-04, 5.9824e-04, 5.6216e-04, 6.2982e-04, 5.6603e-04, 5.6209e-04,
         8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01,
         8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01,
         8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01, 8.3428e-01,
         8.3428e-01, 8.3428e-01, 8.3428e-01],
        [9.9881e-01, 9.9775e-01, 5.1092e-04, 5.4794e-04, 5.1083e-04, 9.9721e-01,
         5.2902e-04, 5.1224e-04, 5.2280e-04, 5.1305e-04, 5.2701e-04, 5.1188e-04,
         5.1340e-04, 5.1789e-04, 9.9666e-01, 5.2543e-04, 6.3789e-04, 9.6206e-01,
         9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01,
         9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01,
         9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337e-01, 9.6337

In [6]:
max_extracted_sentences = int(torch.max(torch.tensor([x.sum() for x in extraction_labels])))
vals, predicted_idx = torch.topk((extraction_probabilities * mask), k=max_extracted_sentences, dim=1)
vals

tensor([[9.9871e-01, 9.9860e-01, 9.9328e-01, 7.0618e-04],
        [9.9881e-01, 9.9775e-01, 9.9721e-01, 9.9666e-01],
        [9.9866e-01, 9.9580e-01, 5.5258e-04, 5.2462e-04],
        [9.9902e-01, 9.9828e-01, 9.9806e-01, 5.5226e-04],
        [9.9894e-01, 9.9876e-01, 5.3652e-04, 5.1733e-04],
        [9.9823e-01, 9.9809e-01, 5.7799e-04, 5.6243e-04],
        [9.9861e-01, 9.9774e-01, 5.4555e-04, 5.2975e-04],
        [9.9899e-01, 9.9816e-01, 9.9814e-01, 5.3069e-04],
        [9.9859e-01, 9.9566e-01, 5.5375e-04, 5.3051e-04],
        [9.9828e-01, 9.9812e-01, 9.9730e-01, 1.1658e-03]],
       grad_fn=<TopkBackward>)

In [7]:
for i, doc in enumerate(documents):
    label_idx = extraction_labels[i].numpy()[:len(doc)]
    label_idx = label_idx.astype(bool)
    
    n_summary_sentences = label_idx.sum()
    
    target = np.array(doc)[label_idx]
    prediction = np.array(doc)[predicted_idx[i]][:n_summary_sentences]
    
    print("\n\n".join(target.tolist()))
    print("\n\n---------------------------\n\n")
    print("\n\n".join(prediction.tolist()))
    
    print("\n\n-----------------------------------------------------")
    print("-----------------------------------------------------")
    print("-----------------------------------------------------\n\n")


one french citizen , one belgian and three malians were killed in the attack in the capital of bamako , said gabriel toure , director of a local hospital .

a north african jihadist group , al - murabitun , claimed responsibility for the attack , according to mauritanian news agency al akhbar . the purported claim came in an audio message in which the group said it carried out the attack in retaliation for the killing of one of its leaders , al akhbar said .

" al - murabitun may be considered a regional competitor to al - qaeda in the islamic maghreb ( aqim ) , " according to the jamestown foundation , a washington - based research and analysis firm . the u.s. state department said in january that al - murabitun is a " newly - formed " militant group that has presence in northern mali .


---------------------------


" al - murabitun may be considered a regional competitor to al - qaeda in the islamic maghreb ( aqim ) , " according to the jamestown foundation , a washington - based r

In [8]:
sentence_embeddings[0][0]

tensor([ 9.1769e-01, -2.0674e-01,  6.3207e-01,  5.6439e-01, -8.3771e-02,
        -4.8105e-01,  2.4024e-02, -2.5747e-01, -1.2952e-01, -3.6122e-01,
        -2.4696e-01, -4.6677e-02, -1.6326e-01,  3.9872e-01,  3.7589e-01,
         1.4049e-01,  6.2072e-02,  4.3766e-01,  1.4464e-01, -9.2281e-02,
         2.7551e-01, -6.0158e-03,  1.7492e-01,  2.5955e-01, -2.9315e-01,
        -1.7080e-01, -4.0563e-01,  4.8684e-02, -9.3265e-01, -7.5062e-02,
         1.1918e-01,  2.8480e-03, -6.3848e-01,  3.3696e-02, -3.0938e-01,
         2.7421e-01,  8.0323e-01, -2.9562e-01, -1.7062e-01, -5.8972e-01,
        -2.6418e-01,  3.9753e-02, -1.1204e-01,  1.6081e-01,  1.3104e-01,
        -4.7816e-01, -1.3782e+00,  1.4592e-01, -4.1722e-01, -1.5556e-01,
        -3.6537e-01,  1.2860e-01,  2.0231e-01,  4.0561e-01,  6.6253e-02,
         5.1574e-01, -3.0309e-01, -4.5073e-01, -2.0610e-01, -2.8797e-01,
         8.9413e-01, -2.0012e-01,  1.3355e-01, -2.2603e-01, -6.2989e-02,
         3.0254e-01,  4.5893e-01,  1.6862e-01, -8.3

In [9]:
sentence_embeddings[0][1]

tensor([ 6.2558e-01,  1.8704e-01,  3.4348e-01,  3.8366e-01,  7.3473e-03,
        -5.4767e-01, -1.0662e-01, -2.7037e-01, -5.4544e-02, -7.5444e-01,
        -2.9372e-01,  7.3500e-03, -1.0518e-01,  6.4475e-01,  5.3719e-01,
         3.2536e-01,  1.5066e-01,  2.4810e-01,  8.9199e-02, -2.4932e-01,
         5.9902e-01, -3.6116e-01,  4.3141e-01,  5.0513e-01, -2.2652e-01,
        -3.6240e-01, -1.9804e-01, -2.0361e-01, -6.6138e-01,  7.9904e-02,
         4.5859e-01, -1.7693e-01, -4.2044e-01,  4.0486e-01, -1.2950e-01,
        -3.1648e-02,  6.7985e-01, -3.8268e-01,  3.4966e-02, -2.6705e-01,
        -1.0670e-01, -2.4639e-01, -1.7908e-01,  1.1528e-01, -3.6503e-02,
        -5.5348e-01, -7.7778e-01, -1.4643e-01, -2.6865e-01, -4.4979e-01,
        -2.6880e-01, -3.5488e-02,  1.3668e-03,  7.0071e-01, -2.8231e-01,
         1.4370e-01, -3.8639e-01, -5.5930e-01, -2.6255e-01, -6.1067e-01,
         6.6340e-01, -1.1970e-01,  1.1543e-01, -5.4215e-01,  1.8875e-01,
         4.2616e-01,  5.0088e-01,  5.0282e-02, -3.2